In [33]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

dataset = pd.read_csv('financial_fraud_detection_dataset.csv')

In [34]:
# Tidying dataset 
dataset = dataset.drop(['transaction_id', 'timestamp', 'sender_account', 'receiver_account', 'fraud_type', 'time_since_last_transaction', 'ip_address', 'device_hash'], axis=1)

# Display all the columns of the dataset 
dataset.head(5)

,amount,transaction_type,merchant_category,location,device_used,is_fraud,spending_deviation_score,velocity_score,geo_anomaly_score,payment_channel
0,343.78,withdrawal,utilities,Tokyo,mobile,False,-0.21,3,0.22,card
1,419.65,withdrawal,online,Toronto,atm,False,-0.14,7,0.96,ACH
2,2773.86,deposit,other,London,pos,False,-1.78,20,0.89,card
3,1666.22,deposit,online,Sydney,pos,False,-0.60,6,0.37,wire_transfer
4,24.43,transfer,utilities,Toronto,mobile,False,0.79,13,0.27,ACH


## Introduction 

Financial fraud  happens when someone takes money or other assets from you illegally, and there are many different types types such as identity, payment, account takeover, etc. Especially due to the growing banking system and the financial technologies, we would like to research and learn more into what could be used to determine whether a transaction is fraud or not, and which category(ies) would be more important than others for detection.

Our goal for this project is to detect financial fraud using machine learning given synthetical data. We would like to be able to detect whether a transaction is fraud or not, and what type of fraud it would be.

## Variable Description

There were a total of 18 variables in the original dataset which we had removed for 10. 

The following variables are the ones we will focus on: 

- amount (USD)
- transaction_type: type of transaction [deposit,  withdrawal, transfer, payment]
- merchant_category: business category involved in transaction [retail, travel, online, utilities, entertainment, etc...]
- location: location from where transaction was initiated
- device_used: device type used [mobile, web, atm, pos]
- is_fraud: boolean flag indicating whether the transaction was fraudulent [true, false]
- spending_deviation_score: deviation from normal spending pattern
- velocity_score: number of transactions over a recent period
- geo_anomaly_score: measure of unusual geographic transaction behaviour [0, 1]
- payment_channel: card, ACH, wire_transfer, UPI


In [35]:
# We want to find the correlation between the features and the target variable 'is_fraud'
